# Section 4: Several Exercises over implementation.

### Imports.

In [1]:

import numpy as np

### Activation Function & Perceptron.

In [2]:

def step(x):
    """Binary step activation function."""
    return int(x >= 0)


def neuron(x, w, b):
    """Single perceptron neuron."""
    return step(np.dot(x, w) + b)

### General AND/OR in one function.

In [3]:

def compute_gate(x_dict, gate_type="and"):
    """
    Generic perceptron-based logic gate handler.
    Args:
        x_dict: dict of {var_name: (value, is_positive)}
        gate_type: "and" or "or"
    """
    values = np.array([v for v, _ in x_dict.values()]) 
    weights = np.array([1 if pos else -1 for _, pos in x_dict.values()])
    L = sum(pos for _, pos in x_dict.values())
    N = len(values)

    bias = -L if gate_type == "and" else -(L - N + 1)
    return neuron(values, weights, bias)


def general_and(x_dict):
    return compute_gate(x_dict, "and")


def general_or(x_dict):
    return compute_gate(x_dict, "or")

### Test This Expression: (¬y ∧ ¬z) ∨ (¬w ∧ x ∧ ¬y) ∨ (¬x ∧ ¬z):

In [4]:

def test_complex_expression():
    print("w x y z | perceptron expected")
    print("-----------------------------")

    for w in [0, 1]:
        for x in [0, 1]:
            for y in [0, 1]:
                for z in [0, 1]:
                    p1 = {"y": (y, False), "z": (z, False)}
                    p2 = {"w": (w, False), "x": (x, True), "y": (y, False)}
                    p3 = {"x": (x, False), "z": (z, False)}
                    p4 = {
                        "p1": (general_and(p1), True),
                        "p2": (general_and(p2), True),
                        "p3": (general_and(p3), True),
                    }

                    perceptron_out = general_or(p4)
                    expected = int(
                        (not y and not z)
                        or (not w and x and not y)
                        or (not x and not z)
                    )

                    print(f"{w} {x} {y} {z} |    {perceptron_out}       {expected}")

test_complex_expression()

w x y z | perceptron expected
-----------------------------
0 0 0 0 |    1       1
0 0 0 1 |    0       0
0 0 1 0 |    1       1
0 0 1 1 |    0       0
0 1 0 0 |    1       1
0 1 0 1 |    1       1
0 1 1 0 |    0       0
0 1 1 1 |    0       0
1 0 0 0 |    1       1
1 0 0 1 |    0       0
1 0 1 0 |    1       1
1 0 1 1 |    0       0
1 1 0 0 |    1       1
1 1 0 1 |    0       0
1 1 1 0 |    0       0
1 1 1 1 |    0       0


### Defining Xor Over 2 element. Xor over N-elements Recursively and Iteratively using Divide & Conquer.

In [5]:
def xor_basic(x):
    """Two-input XOR using perceptrons."""
    n1 = neuron(x, np.array([1, -1]), -1)
    n2 = neuron(x, np.array([-1, 1]), -1)
    return neuron(np.array([n1, n2]), np.array([1, 1]), -1)


def xor_divide_conquer(inputs):
    """Recursive XOR for any number of inputs."""
    if len(inputs) == 1:
        return inputs[0]
    if len(inputs) == 2:
        return xor_basic(np.array(inputs))

    mid = len(inputs) // 2
    left = xor_divide_conquer(inputs[:mid])
    right = xor_divide_conquer(inputs[mid:])
    return xor_basic(np.array([left, right]))


def xor(arr):
    """Simple XOR on a NumPy array of two elements."""
    return arr[0] ^ arr[1]


def xor_divide_conquer_iterative(inputs):
    """Iterative divide-and-conquer XOR for any number of inputs (handles odd lengths)."""
    # Copy to avoid modifying the original list
    current = np.array(inputs, dtype=int)

    # Keep combining until one element remains
    while len(current) > 1:
        next_level = []

        # Process pairs
        for i in range(0, len(current) - 1, 2):
            next_level.append(xor(np.array([current[i], current[i + 1]])))

        # If odd number of elements, carry last one forward
        if len(current) % 2 == 1:
            next_level.append(current[-1])

        # Move to next level
        current = np.array(next_level, dtype=int)

    return current[0]

### Xor over N-elements Linearly w/ Test.

In [6]:
def test_xor_basic():
    print("w x y z | perceptron expected")
    print("--------------------------")
    for w_val in [0, 1]:
        for x_val in [0, 1]:
            for y_val in [0, 1]:
                for z_val in [0, 1]:
                    ls = [w_val, x_val, y_val, z_val]
                    while len(ls) > 1:
                        vals = [ls.pop(0), ls.pop(0)]
                        out = xor_basic(np.array(vals))
                        ls.insert(0, out)
                    expected = int(x_val ^ y_val ^ z_val ^ w_val)

                    print(f"{w_val} {x_val} {y_val} {z_val} |    {ls[0]}       {expected}")
                    
test_xor_basic()

w x y z | perceptron expected
--------------------------
0 0 0 0 |    0       0
0 0 0 1 |    1       1
0 0 1 0 |    1       1
0 0 1 1 |    0       0
0 1 0 0 |    1       1
0 1 0 1 |    0       0
0 1 1 0 |    0       0
0 1 1 1 |    1       1
1 0 0 0 |    1       1
1 0 0 1 |    0       0
1 0 1 0 |    0       0
1 0 1 1 |    1       1
1 1 0 0 |    0       0
1 1 0 1 |    1       1
1 1 1 0 |    1       1
1 1 1 1 |    0       0


### Testing The Divide & Conquer Approach.

In [7]:
def test_xor_multi():
    print("w x y | iterative recursive expected")
    print("------------------------------------")
    for w in [0, 1]:
        for x in [0, 1]:
            for y in [0, 1]:
                vals = [w, x, y]
                out = xor_divide_conquer_iterative(vals)
                out2 = xor_divide_conquer(vals) 
                print(f"{w} {x} {y} |   {out}           {out2}       {w ^ x ^ y}")

test_xor_multi()


w x y | iterative recursive expected
------------------------------------
0 0 0 |   0           0       0
0 0 1 |   1           1       1
0 1 0 |   1           1       1
0 1 1 |   0           0       0
1 0 0 |   1           1       1
1 0 1 |   0           0       0
1 1 0 |   0           0       0
1 1 1 |   1           1       1
